Computational modeling in python, SoSe2022 


# Timing methods

timing will somehow access the system clock so the results and their granularity will be platform dependent. 

In [45]:
import numpy as np


def function1(x):
    y = x*(x - 3)*(x + 3)
    return y


In [46]:
def integrate_MC1D_direct(N,myfunc,a,b):
    
    x_rand = (b-a)*np.random.random(N)+a
    y_rand = myfunc(x_rand)
    
    integral = y_rand.sum()*(b-a)/N
    
    return integral

## Timing with magic functions

In [47]:
%%time
# timing of the complete cell

N = 1000000
a = -4
b = 4

integrate_MC1D_direct(N,function1,a,b)


CPU times: user 14.2 ms, sys: 12 ms, total: 26.1 ms
Wall time: 22.1 ms


0.1127219525844164

In [48]:
# timing of a single command

N = 1000000
a = -4
b = 4

%time integrate_MC1D_direct(N,function1,a,b)


CPU times: user 14.7 ms, sys: 12.8 ms, total: 27.5 ms
Wall time: 24.5 ms


-0.1015367054734847

# Using the `time` module

In [49]:
import time
# there is also the timeit.timer() function which defaults to time.perf_counter() in python 3

N = 1000000
a = -4
b = 4

# system time (undefined reference, so only differences matter)
tstart = time.perf_counter()

I = integrate_MC1D_direct(N,function1,a,b)

tend = time.perf_counter()

print(tend-tstart, 'seconds')

0.025491954000244732 seconds


In [50]:

N = 1000000
a = -4
b = 4

# runtime of the current process (user + system, not counting sleep time)
tstart = time.process_time()

I = integrate_MC1D_direct(N,function1,a,b)

tend = time.process_time()

print(tend-tstart, ' seconds')

0.027945783999999918  seconds


### Timing should be done several times to obtain reliable results

In [51]:

N = 1000000
a = -4
b = 4
nperf = 20

ttotal = 0.0

for i in range(nperf):
    tstart = time.perf_counter()
    I = integrate_MC1D_direct(N,function1,a,b)
    tend = time.perf_counter()
    ttotal = ttotal + tend-tstart
    

print(ttotal/nperf, ' seconds on average')

0.014169931750052456  seconds on average


In [52]:
%%timeit -n 3 -r 4

I = integrate_MC1D_direct(N,function1,a,b)

13.3 ms ± 1.79 ms per loop (mean ± std. dev. of 4 runs, 3 loops each)


# Profiling code with `%prun`

Finding parts of code that are slow (or fast) 

In [53]:
def minmax1(myarray):
       
    mymin1 = min(myarray)
    mymax1 = max(myarray)
    return mymin1, mymax1
    
    
def minmax2(mayarray):
    mymin2 = myarray.min()
    mymax2 = myarray.max()
    return mymin2, mymax2

In [54]:
N = 10000000
myarray = np.random.random(N)

# there is a sgnificant difference in  min(myarray) and myarray.min() etc. 
%prun mymin1, mymax1 = minmax1(myarray)

In [55]:
# there is a sgnificant difference in  min(myarray) and myarray.min() etc. 
%prun mymin2, mymax2 = minmax2(myarray)

# Profiling with `cProfile`:

In [56]:
import cProfile

cProfile.run('integrate_MC1D_direct(100000,function1,-4,4)')

         9 function calls in 0.005 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.002    0.002 1291826261.py:4(function1)
        1    0.001    0.001    0.005    0.005 4147265916.py:1(integrate_MC1D_direct)
        1    0.000    0.000    0.005    0.005 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _methods.py:46(_sum)
        1    0.000    0.000    0.005    0.005 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.001    0.001    0.001    0.001 {method 'random' of 'numpy.random.mtrand.RandomState' objects}
        1    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.000    0.000    0.000    0.000 {method 'sum' of 'numpy.ndarray' objects}




If you have a python script you can run the script using `cProfile`:


`python -m cProfile myscript.py`

on the command line.



# Profiling with IPython extensions:

`conda install line_profiler`

`conda install memory_profiler`


In [62]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [67]:
%lprun -f integrate_MC1D_direct integrate_MC1D_direct(100000,function1,-4,4)
#lprun?

In [68]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [69]:
%memit integrate_MC1D_direct(100000,function1,-4,4)

peak memory: 167.80 MiB, increment: 0.00 MiB


`pip install py-heat-magic`

`heat` requires everything defined in the cell that is to be profiled

In [71]:
%load_ext heat

ModuleNotFoundError: No module named 'heat'

In [72]:
%%heat 

import numpy as np

def function1(x):
    y = x*(x - 3)*(x + 3)
    return y


def integrate_MC1D_direct(N,myfunc,a,b):
    
    x_rand = (b-a)*np.random.random(N)+a
    y_rand = myfunc(x_rand)
    
    integral = y_rand.sum()*(b-a)/N
    
    return integral

integrate_MC1D_direct(1000,function1,-4,4)

UsageError: Cell magic `%%heat` not found.
